In [1]:
%load_ext autoreload
%autoreload 2

from common import add_workspace_to_path

add_workspace_to_path()

In [2]:
from cloud.dependencies.clients import resolve_gmail_client


gm = resolve_gmail_client()